In [1]:
import tensorflow as tf
import os
import glob
import numpy as np
import math

In [2]:
tf.app.flags.DEFINE_string('train_data_dir', '/Users/Olivier/c399project/GTSRB/Train',
                           """Path to the Traffic data directory.""")
tf.app.flags.DEFINE_string('test_data_dir', '/Users/Olivier/c399project/GTSRB/Test',
                           """Path to save training info.""")
tf.app.flags.DEFINE_string('train_dir', '/Users/Olivier/c399project',
                           """Path to save training info.""")

In [3]:
sess = tf.InteractiveSession()

In [4]:
FLAGS = tf.app.flags.FLAGS
train_data_dir = FLAGS.train_data_dir
test_data_dir = FLAGS.test_data_dir
train_dir = FLAGS.train_dir

size = 32,32
classes = 43
val_size = 12630
batch_size = 100
max_steps = 2000

X = tf.placeholder(tf.float32, name = "input")
T = tf.placeholder(tf.float32)
target_Class = tf.placeholder(tf.int32)
is_training = tf.placeholder(tf.bool, [], name='is_training')

In [5]:
def read_and_decode(filename_queue):
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)
    features = tf.parse_single_example(
        serialized_example,
        features={
            'image/encoded': tf.FixedLenFeature([], tf.string),
            'image/class/label': tf.FixedLenFeature([], tf.int64),
        })
    label = tf.cast(features['image/class/label'], tf.int32) - 1
    
    reshaped_image = tf.image.decode_jpeg(features['image/encoded'], channels = 3)
    method=0
    resized_image = tf.image.resize_images(reshaped_image, (size[0], size[1]), method = 0)
    return resized_image, label

In [6]:
def distorted_inputs(batch_size, num_epochs):

    subset = "train"
    
    tf_record_pattern = os.path.join(FLAGS.train_dir + '/GTSRB', '%s-*' % subset)
    data_files = tf.gfile.Glob(tf_record_pattern)
    
    
    filename_queue = tf.train.string_input_producer(data_files, shuffle=True)
    image, label = read_and_decode(filename_queue)
    distorted_image = tf.image.random_brightness(image, max_delta=63)
    distorted_image = tf.image.random_contrast(distorted_image, lower=0.2, upper=1.8)
    normalized_image = tf.image.per_image_whitening(distorted_image)
    
    # Shuffle the examples and collect them into batch_size batches.
    # (Internally uses a RandomShuffleQueue.)
    # We run this in two threads to avoid being a bottleneck.
    
    images, sparse_labels = tf.train.shuffle_batch(
        [normalized_image, label], batch_size=batch_size, num_threads=2,
        capacity=1000 + 3 * batch_size,
        # Ensures a minimum amount of shuffling of examples.
        min_after_dequeue=1000)
    return images, sparse_labels

In [7]:
def inputs(batch_size, num_epochs):

    subset = "validation"
    
    tf_record_pattern = os.path.join(FLAGS.train_dir + '/GTSRB', '%s-*' % subset)
    data_files = tf.gfile.Glob(tf_record_pattern)
    
    filename_queue = tf.train.string_input_producer(data_files, shuffle=False, capacity=1)#num_epochs=num_epochs)
    image, label = read_and_decode(filename_queue)
    normalized_image = tf.image.per_image_whitening(image)
    
    images, sparse_labels = tf.train.batch(
        [normalized_image, label], batch_size=batch_size, num_threads=2,
        capacity=1000 + 3 * batch_size)
    return images, sparse_labels

In [8]:
def weight_variable(shape):
    #He et al
    stddev=math.sqrt(2.0/np.prod(shape[:3]))
    #print stddev
    initial = tf.random_normal(shape, stddev=stddev)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.01, shape=shape)
    return tf.Variable(initial)

In [9]:
filters_conv1 = 16
dropout = 0.50

#Conv 1
W_conv1 = weight_variable([3, 3, 3, filters_conv1])
b_conv1 = bias_variable([filters_conv1])
conv1 = tf.nn.conv2d(X, W_conv1, strides =[1, 1, 1, 1], padding = 'SAME')

#Relu 1
relu1 = tf.nn.relu(conv1 + b_conv1)

#Dropout first layer
#if is_training is not None:
#     relu1 = tf.nn.dropout(relu1, dropout)

#Batch norm 1
batch_norm1 = tf.contrib.layers.batch_norm(relu1, is_training= is_training, updates_collections= None)

#Max pool 1
max_pool1 = tf.nn.max_pool(batch_norm1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

filters_conv2 = 32

#Conv 2
W_conv2 = weight_variable([3, 3, filters_conv1, filters_conv2])
b_conv2 = bias_variable([filters_conv2])
conv2 = tf.nn.conv2d(max_pool1, W_conv2, strides =[1,1,1,1], padding = 'SAME')

#Relu 2
relu2 = tf.nn.relu(conv2 + b_conv2)

#Dropout second layer
#if is_training is not None:
#     relu2 = tf.nn.dropout(relu2, dropout)

#Batch norm 2
batch_norm2 = tf.contrib.layers.batch_norm(relu2, is_training= is_training, updates_collections= None)

#Max pool 2
max_pool2 = tf.nn.max_pool(batch_norm2, ksize=[1,2,2,1], strides =[1,2,2,1],padding='SAME')

filters_conv3 = 64

#Conv 3
W_conv3 = weight_variable([3, 3, filters_conv2, filters_conv3])
b_conv3 = bias_variable([filters_conv3])
conv3 = tf.nn.conv2d(max_pool2, W_conv3, strides =[1,1,1,1], padding = 'SAME')

#Relu 3
relu3 = tf.nn.relu(conv3 + b_conv3)

#Dropout last layer
#if is_training is not None:
#     relu3 = tf.nn.dropout(relu3, dropout)

#Batch norm 3
batch_norm3 = tf.contrib.layers.batch_norm(relu3, is_training= is_training, updates_collections= None)

#Max pool 3
max_pool3 = tf.nn.max_pool(batch_norm3, ksize=[1,2,2,1], strides =[1,2,2,1],padding='SAME')


#Local 
filters_fc1 =  classes


#Reduced to 3 x 3
W_fc1 = weight_variable([4 * 4 * filters_conv3, filters_fc1])
b_fc1 = bias_variable([filters_fc1])

max_pool3_flat = tf.reshape(max_pool3, [-1, 4 * 4 * filters_conv3])
fc1 = tf.matmul(max_pool3_flat,W_fc1) + b_fc1 #593, 10

#Y=tf.nn.softmax(fc1, name="output")

In [10]:
T = tf.cast(tf.one_hot(indices = target_Class, depth = classes, on_value=1, off_value=0,axis=-1),tf.float32)
#to_sum = (T * tf.log(Y + 1e-10))
#cross_entropy = tf.reduce_mean(-tf.reduce_sum(to_sum, reduction_indices=1))

regularizers = (tf.nn.l2_loss(W_fc1) + tf.nn.l2_loss(b_fc1)) 
hinge_loss = tf.contrib.losses.hinge_loss(fc1, T)


final_loss = tf.reduce_mean(tf.cast(hinge_loss + regularizers * 5e-4, tf.float32))
#final_loss = tf.reduce_mean(tf.cast(hinge_loss, tf.float32))
loss_summary = tf.scalar_summary('final_loss', final_loss)

#loss_summary = tf.scalar_summary('cross_entropy', cross_entropy)

correct_prediction = tf.equal(tf.argmax(fc1,1), tf.argmax(T,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
accuracy_summary = tf.scalar_summary('accuracy', accuracy)

train_step = tf.train.AdamOptimizer(5e-3).minimize(final_loss)#1e-4
saver = tf.train.Saver()
summary_op = tf.merge_summary([loss_summary, accuracy_summary])
summary_writer = tf.train.SummaryWriter(os.path.join(train_dir, 'Log'), sess.graph)

In [11]:
train_input = distorted_inputs(batch_size,1000)
val_input = inputs(12630,1000)

In [12]:
sess.run(tf.initialize_all_variables())
sess.run(tf.initialize_local_variables())
tf.train.start_queue_runners(sess=sess)

[<Thread(Thread-5, started daemon 4685041664)>,
 <Thread(Thread-6, started daemon 4689248256)>,
 <Thread(Thread-7, started daemon 4693454848)>,
 <Thread(Thread-8, started daemon 4697661440)>,
 <Thread(Thread-9, started daemon 4701868032)>,
 <Thread(Thread-10, started daemon 4706074624)>]

# Benchmark (No Regularization lr = 1e-4)

In [13]:
for i in range(max_steps):
    
    train_batch = sess.run(train_input)
    summary_str, _ = sess.run([summary_op, train_step], feed_dict={
            X:train_batch[0], target_Class:train_batch[1], is_training: True})
    summary_writer.add_summary(summary_str, i)    
    if i % 100 == 0:
        loss = sess.run(cross_entropy,feed_dict={X:train_batch[0], target_Class:train_batch[1], is_training: False})
        print 'Step: ',i,'. Loss: ',loss
        #if loss < 0.05 :
        #    saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))
        #    break
    if i % 1000 == 0 or (i + 1) == max_steps:
        saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))

Step:  0 . Loss:  3.7536
Step:  100 . Loss:  3.24122
Step:  200 . Loss:  2.89084
Step:  300 . Loss:  2.68005
Step:  400 . Loss:  2.4579
Step:  500 . Loss:  2.36892
Step:  600 . Loss:  2.24055
Step:  700 . Loss:  2.088
Step:  800 . Loss:  1.81157
Step:  900 . Loss:  1.72236
Step:  1000 . Loss:  1.49664
Step:  1100 . Loss:  1.42123
Step:  1200 . Loss:  1.24232
Step:  1300 . Loss:  1.08342
Step:  1400 . Loss:  1.07183
Step:  1500 . Loss:  0.929261
Step:  1600 . Loss:  0.801677
Step:  1700 . Loss:  0.737931
Step:  1800 . Loss:  0.721035
Step:  1900 . Loss:  0.58069


In [14]:
val_batch = sess.run(val_input)
acc = sess.run(accuracy, feed_dict={X: val_batch[0], target_Class:val_batch[1], is_training: False})
print acc

0.859778


# No Regularization (lr = 1e-3)

In [19]:
for i in range(max_steps):
    
    train_batch = sess.run(train_input)
    summary_str, _ = sess.run([summary_op, train_step], feed_dict={
            X:train_batch[0], target_Class:train_batch[1], is_training: True})
    summary_writer.add_summary(summary_str, i)    
    if i % 100 == 0:
        loss = sess.run(cross_entropy,feed_dict={X:train_batch[0], target_Class:train_batch[1], is_training: False})
        print 'Step: ',i,'. Loss: ',loss
        #if loss < 0.05 :
        #    saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))
        #    break
    if i % 1000 == 0 or (i + 1) == max_steps:
        saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))

Step:  0 . Loss:  3.31417
Step:  100 . Loss:  2.61827
Step:  200 . Loss:  2.22712
Step:  300 . Loss:  1.97276
Step:  400 . Loss:  1.78759
Step:  500 . Loss:  1.35738
Step:  600 . Loss:  1.32299
Step:  700 . Loss:  0.759078
Step:  800 . Loss:  0.616521
Step:  900 . Loss:  0.464506
Step:  1000 . Loss:  0.449309
Step:  1100 . Loss:  0.302529
Step:  1200 . Loss:  0.233699
Step:  1300 . Loss:  0.234065
Step:  1400 . Loss:  0.23437
Step:  1500 . Loss:  0.0814788
Step:  1600 . Loss:  0.0802397
Step:  1700 . Loss:  0.0812941
Step:  1800 . Loss:  0.0619496
Step:  1900 . Loss:  0.0243947


In [20]:
val_batch = sess.run(val_input)
acc = sess.run(accuracy, feed_dict={X: val_batch[0], target_Class:val_batch[1], is_training: False})
print acc

0.926207


# Attempt 2, No Regularization (lr = 1e-3)

In [56]:
for i in range(max_steps):
    
    train_batch = sess.run(train_input)
    summary_str, _ = sess.run([summary_op, train_step], feed_dict={
            X:train_batch[0], target_Class:train_batch[1], is_training: True})
    summary_writer.add_summary(summary_str, i)    
    if i % 100 == 0:
        loss = sess.run(cross_entropy,feed_dict={X:train_batch[0], target_Class:train_batch[1], is_training: False})
        print 'Step: ',i,'. Loss: ',loss
        #if loss < 0.05 :
        #    saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))
        #    break
    if i % 1000 == 0 or (i + 1) == max_steps:
        saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))

Step:  0 . Loss:  3.20159
Step:  100 . Loss:  2.25153
Step:  200 . Loss:  2.35871
Step:  300 . Loss:  2.20054
Step:  400 . Loss:  1.99434
Step:  500 . Loss:  1.84756
Step:  600 . Loss:  1.29041
Step:  700 . Loss:  1.20245
Step:  800 . Loss:  1.05675
Step:  900 . Loss:  0.837313
Step:  1000 . Loss:  0.691346
Step:  1100 . Loss:  0.406833
Step:  1200 . Loss:  0.380381
Step:  1300 . Loss:  0.274885
Step:  1400 . Loss:  0.185056
Step:  1500 . Loss:  0.112368
Step:  1600 . Loss:  0.111585
Step:  1700 . Loss:  0.0823438
Step:  1800 . Loss:  0.0914605
Step:  1900 . Loss:  0.0601741


In [57]:
val_batch = sess.run(val_input)
acc = sess.run(accuracy, feed_dict={X: val_batch[0], target_Class:val_batch[1], is_training: False})
print acc

0.930325


# No Regularization (lr = 1e-2)

In [ ]:
for i in range(max_steps):
    
    train_batch = sess.run(train_input)
    summary_str, _ = sess.run([summary_op, train_step], feed_dict={
            X:train_batch[0], target_Class:train_batch[1], is_training: True})
    summary_writer.add_summary(summary_str, i)    
    if i % 100 == 0:
        loss = sess.run(cross_entropy,feed_dict={X:train_batch[0], target_Class:train_batch[1], is_training: False})
        print 'Step: ',i,'. Loss: ',loss
        #if loss < 0.05 :
        #    saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))
        #    break
    if i % 1000 == 0 or (i + 1) == max_steps:
        saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))

# Dropout = 0.5 (all three layers), lr = 1e-4

In [28]:
for i in range(max_steps):
    
    train_batch = sess.run(train_input)
    summary_str, _ = sess.run([summary_op, train_step], feed_dict={
            X:train_batch[0], target_Class:train_batch[1], is_training: True})
    summary_writer.add_summary(summary_str, i)    
    if i % 100 == 0:
        loss = sess.run(cross_entropy,feed_dict={X:train_batch[0], target_Class:train_batch[1], is_training: False})
        print 'Step: ',i,'. Loss: ',loss
        if loss < 0.05 :
            saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))
            break
    if i % 1000 == 0 or (i + 1) == max_steps:
        saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))

Step:  0 . Loss:  4.91831
Step:  100 . Loss:  4.0078
Step:  200 . Loss:  3.33719
Step:  300 . Loss:  2.56912
Step:  400 . Loss:  2.55848
Step:  500 . Loss:  1.80295
Step:  600 . Loss:  1.70359
Step:  700 . Loss:  1.49646
Step:  800 . Loss:  1.28805
Step:  900 . Loss:  1.46846
Step:  1000 . Loss:  1.37136
Step:  1100 . Loss:  1.39738
Step:  1200 . Loss:  1.23874
Step:  1300 . Loss:  1.18244
Step:  1400 . Loss:  1.08036
Step:  1500 . Loss:  1.18825
Step:  1600 . Loss:  1.02413
Step:  1700 . Loss:  1.07764
Step:  1800 . Loss:  0.924771
Step:  1900 . Loss:  0.979659


In [29]:
val_batch = sess.run(val_input)
acc = sess.run(accuracy, feed_dict={X: val_batch[0], target_Class:val_batch[1], is_training: False})
print acc

0.649248


# Dropout = 0.75 (all three layers), lr = 1e-4

In [34]:
for i in range(max_steps):
    
    train_batch = sess.run(train_input)
    summary_str, _ = sess.run([summary_op, train_step], feed_dict={
            X:train_batch[0], target_Class:train_batch[1], is_training: True})
    summary_writer.add_summary(summary_str, i)    
    if i % 100 == 0:
        loss = sess.run(cross_entropy,feed_dict={X:train_batch[0], target_Class:train_batch[1], is_training: False})
        print 'Step: ',i,'. Loss: ',loss
        if loss < 0.05 :
            saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))
            break
    if i % 1000 == 0 or (i + 1) == max_steps:
        saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))

Step:  0 . Loss:  3.91304
Step:  100 . Loss:  3.51453
Step:  200 . Loss:  2.88305
Step:  300 . Loss:  2.23679
Step:  400 . Loss:  1.88749
Step:  500 . Loss:  1.65306
Step:  600 . Loss:  1.47248
Step:  700 . Loss:  1.26717
Step:  800 . Loss:  1.05781
Step:  900 . Loss:  1.04965
Step:  1000 . Loss:  1.19831
Step:  1100 . Loss:  0.841833
Step:  1200 . Loss:  0.960767
Step:  1300 . Loss:  0.834251
Step:  1400 . Loss:  0.822843
Step:  1500 . Loss:  0.728744
Step:  1600 . Loss:  0.565386
Step:  1700 . Loss:  0.641309
Step:  1800 . Loss:  0.582814
Step:  1900 . Loss:  0.552345


In [35]:
val_batch = sess.run(val_input)
acc = sess.run(accuracy, feed_dict={X: val_batch[0], target_Class:val_batch[1], is_training: False})
print acc

0.782344


# Dropout = 0.75 (all three layers), lr = 1e-3

In [13]:
for i in range(max_steps):
    
    train_batch = sess.run(train_input)
    summary_str, _ = sess.run([summary_op, train_step], feed_dict={
            X:train_batch[0], target_Class:train_batch[1], is_training: True})
    summary_writer.add_summary(summary_str, i)    
    if i % 100 == 0:
        loss = sess.run(cross_entropy,feed_dict={X:train_batch[0], target_Class:train_batch[1], is_training: False})
        print 'Step: ',i,'. Loss: ',loss
        if loss < 0.05 :
            saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))
            break
    if i % 1000 == 0 or (i + 1) == max_steps:
        saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))

Step:  0 . Loss:  3.60022
Step:  100 . Loss:  1.82001
Step:  200 . Loss:  1.18199
Step:  300 . Loss:  1.04527
Step:  400 . Loss:  1.06241
Step:  500 . Loss:  0.593265
Step:  600 . Loss:  0.591066
Step:  700 . Loss:  0.466514
Step:  800 . Loss:  0.304028
Step:  900 . Loss:  0.26783
Step:  1000 . Loss:  0.236989
Step:  1100 . Loss:  0.185212
Step:  1200 . Loss:  0.126802
Step:  1300 . Loss:  0.135735
Step:  1400 . Loss:  0.0924672
Step:  1500 . Loss:  0.115159
Step:  1600 . Loss:  0.0792972
Step:  1700 . Loss:  0.107193
Step:  1800 . Loss:  0.153237
Step:  1900 . Loss:  0.0378849


In [14]:
val_batch = sess.run(val_input)
acc = sess.run(accuracy, feed_dict={X: val_batch[0], target_Class:val_batch[1], is_training: False})
print acc

0.903642


# Dropout = 0.75 (all three layers), lr = 5e-3

In [18]:
for i in range(max_steps):
    
    train_batch = sess.run(train_input)
    summary_str, _ = sess.run([summary_op, train_step], feed_dict={
            X:train_batch[0], target_Class:train_batch[1], is_training: True})
    summary_writer.add_summary(summary_str, i)    
    if i % 100 == 0:
        loss = sess.run(cross_entropy,feed_dict={X:train_batch[0], target_Class:train_batch[1], is_training: False})
        print 'Step: ',i,'. Loss: ',loss
        if loss < 0.05 :
            saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))
            break
    if i % 1000 == 0 or (i + 1) == max_steps:
        saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))

Step:  0 . Loss:  4.82204
Step:  100 . Loss:  1.87974
Step:  200 . Loss:  2.05365
Step:  300 . Loss:  0.886548
Step:  400 . Loss:  1.32419
Step:  500 . Loss:  0.831887
Step:  600 . Loss:  0.64722
Step:  700 . Loss:  0.105297
Step:  800 . Loss:  0.366433
Step:  900 . Loss:  0.393805
Step:  1000 . Loss:  0.849938
Step:  1100 . Loss:  0.192251
Step:  1200 . Loss:  0.458831
Step:  1300 . Loss:  0.529752
Step:  1400 . Loss:  0.170316
Step:  1500 . Loss:  0.183271
Step:  1600 . Loss:  0.223511
Step:  1700 . Loss:  0.118869
Step:  1800 . Loss:  0.282904
Step:  1900 . Loss:  0.304339


In [19]:
val_batch = sess.run(val_input)
acc = sess.run(accuracy, feed_dict={X: val_batch[0], target_Class:val_batch[1], is_training: False})
print acc

0.890024


# Dropout = 0.75 (only first layer), lr = 1e-4

In [13]:
for i in range(max_steps):
    
    train_batch = sess.run(train_input)
    summary_str, _ = sess.run([summary_op, train_step], feed_dict={
            X:train_batch[0], target_Class:train_batch[1], is_training: True})
    summary_writer.add_summary(summary_str, i)    
    if i % 100 == 0:
        loss = sess.run(cross_entropy,feed_dict={X:train_batch[0], target_Class:train_batch[1], is_training: False})
        print 'Step: ',i,'. Loss: ',loss
        if loss < 0.05 :
            saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))
            break
    if i % 1000 == 0 or (i + 1) == max_steps:
        saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))

Step:  0 . Loss:  3.83697
Step:  100 . Loss:  3.14492
Step:  200 . Loss:  3.01874
Step:  300 . Loss:  2.72159
Step:  400 . Loss:  2.43664
Step:  500 . Loss:  2.25104
Step:  600 . Loss:  1.85186
Step:  700 . Loss:  1.72528
Step:  800 . Loss:  1.7278
Step:  900 . Loss:  1.46431
Step:  1000 . Loss:  1.34019
Step:  1100 . Loss:  1.12854
Step:  1200 . Loss:  1.09007
Step:  1300 . Loss:  1.05348
Step:  1400 . Loss:  0.960848
Step:  1500 . Loss:  0.754366
Step:  1600 . Loss:  0.619626
Step:  1700 . Loss:  0.687439
Step:  1800 . Loss:  0.523773
Step:  1900 . Loss:  0.50295


In [14]:
val_batch = sess.run(val_input)
acc = sess.run(accuracy, feed_dict={X: val_batch[0], target_Class:val_batch[1], is_training: False})
print acc

0.868725


# Dropout = 0.75 (first layer), lr = 5e-4

In [31]:
for i in range(max_steps):
    
    train_batch = sess.run(train_input)
    summary_str, _ = sess.run([summary_op, train_step], feed_dict={
            X:train_batch[0], target_Class:train_batch[1], is_training: True})
    summary_writer.add_summary(summary_str, i)    
    if i % 100 == 0:
        loss = sess.run(cross_entropy,feed_dict={X:train_batch[0], target_Class:train_batch[1], is_training: False})
        print 'Step: ',i,'. Loss: ',loss
        #if loss < 0.05 :
        #    saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))
        #    break
    if i % 1000 == 0 or (i + 1) == max_steps:
        saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))

Step:  0 . Loss:  3.36925
Step:  100 . Loss:  2.3945
Step:  200 . Loss:  2.05912
Step:  300 . Loss:  1.89554
Step:  400 . Loss:  1.65967
Step:  500 . Loss:  1.54105
Step:  600 . Loss:  1.24334
Step:  700 . Loss:  1.25968
Step:  800 . Loss:  0.912741
Step:  900 . Loss:  0.807883
Step:  1000 . Loss:  0.694755
Step:  1100 . Loss:  0.515607
Step:  1200 . Loss:  0.355836
Step:  1300 . Loss:  0.340719
Step:  1400 . Loss:  0.330222
Step:  1500 . Loss:  0.304393
Step:  1600 . Loss:  0.208659
Step:  1700 . Loss:  0.171567
Step:  1800 . Loss:  0.155338
Step:  1900 . Loss:  0.188009


In [32]:
val_batch = sess.run(val_input)
acc = sess.run(accuracy, feed_dict={X: val_batch[0], target_Class:val_batch[1], is_training: False})
print acc

0.914489


# Dropout = 0.75 (first layer), lr = 1e-3

In [13]:
for i in range(max_steps):
    
    train_batch = sess.run(train_input)
    summary_str, _ = sess.run([summary_op, train_step], feed_dict={
            X:train_batch[0], target_Class:train_batch[1], is_training: True})
    summary_writer.add_summary(summary_str, i)    
    if i % 100 == 0:
        loss = sess.run(cross_entropy,feed_dict={X:train_batch[0], target_Class:train_batch[1], is_training: False})
        print 'Step: ',i,'. Loss: ',loss
        #if loss < 0.05 :
        #    saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))
        #    break
    if i % 1000 == 0 or (i + 1) == max_steps:
        saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))

Step:  0 . Loss:  3.0932
Step:  100 . Loss:  1.85392
Step:  200 . Loss:  1.48162
Step:  300 . Loss:  1.38833
Step:  400 . Loss:  1.1526
Step:  500 . Loss:  0.870631
Step:  600 . Loss:  0.738435
Step:  700 . Loss:  0.413413
Step:  800 . Loss:  0.364456
Step:  900 . Loss:  0.300523
Step:  1000 . Loss:  0.312692
Step:  1100 . Loss:  0.239826
Step:  1200 . Loss:  0.197565
Step:  1300 . Loss:  0.13111
Step:  1400 . Loss:  0.0945244
Step:  1500 . Loss:  0.0694274
Step:  1600 . Loss:  0.0544139
Step:  1700 . Loss:  0.0389439
Step:  1800 . Loss:  0.0358317
Step:  1900 . Loss:  0.0524417


In [14]:
val_batch = sess.run(val_input)
acc = sess.run(accuracy, feed_dict={X: val_batch[0], target_Class:val_batch[1], is_training: False})
print acc

0.937767


# ATTEMPT 2, Dropout = 0.75 (first layer), lr = 1e-3 

In [49]:
for i in range(max_steps):
    
    train_batch = sess.run(train_input)
    summary_str, _ = sess.run([summary_op, train_step], feed_dict={
            X:train_batch[0], target_Class:train_batch[1], is_training: True})
    summary_writer.add_summary(summary_str, i)    
    if i % 100 == 0:
        loss = sess.run(cross_entropy,feed_dict={X:train_batch[0], target_Class:train_batch[1], is_training: False})
        print 'Step: ',i,'. Loss: ',loss
        #if loss < 0.05 :
        #    saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))
        #    break
    if i % 1000 == 0 or (i + 1) == max_steps:
        saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))

Step:  0 . Loss:  3.24065
Step:  100 . Loss:  2.30655
Step:  200 . Loss:  2.10576
Step:  300 . Loss:  1.85185
Step:  400 . Loss:  1.36072
Step:  500 . Loss:  1.25371
Step:  600 . Loss:  0.878236
Step:  700 . Loss:  0.633516
Step:  800 . Loss:  0.546105
Step:  900 . Loss:  0.476141
Step:  1000 . Loss:  0.311988
Step:  1100 . Loss:  0.284903
Step:  1200 . Loss:  0.221223
Step:  1300 . Loss:  0.0884746
Step:  1400 . Loss:  0.131383
Step:  1500 . Loss:  0.105534
Step:  1600 . Loss:  0.0514172
Step:  1700 . Loss:  0.0387598
Step:  1800 . Loss:  0.0319892
Step:  1900 . Loss:  0.0649666


In [51]:
val_batch = sess.run(val_input)
acc = sess.run(accuracy, feed_dict={X: val_batch[0], target_Class:val_batch[1], is_training: False})
print acc

0.921615


# Dropout = 0.75 (first layer), lr = 1e-2

In [18]:
for i in range(max_steps):
    
    train_batch = sess.run(train_input)
    summary_str, _ = sess.run([summary_op, train_step], feed_dict={
            X:train_batch[0], target_Class:train_batch[1], is_training: True})
    summary_writer.add_summary(summary_str, i)    
    if i % 100 == 0:
        loss = sess.run(cross_entropy,feed_dict={X:train_batch[0], target_Class:train_batch[1], is_training: False})
        print 'Step: ',i,'. Loss: ',loss
        #if loss < 0.05 :
        #    saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))
        #    break
    if i % 1000 == 0 or (i + 1) == max_steps:
        saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))

Step:  0 . Loss:  8.32478
Step:  100 . Loss:  12.2061
Step:  200 . Loss:  3.7615
Step:  300 . Loss:  3.09903
Step:  400 . Loss:  1.20554
Step:  500 . Loss:  1.49422
Step:  600 . Loss:  1.76923
Step:  700 . Loss:  0.981331
Step:  800 . Loss:  0.677316
Step:  900 . Loss:  0.278588
Step:  1000 . Loss:  0.156135
Step:  1100 . Loss:  0.489705
Step:  1200 . Loss:  0.196344
Step:  1300 . Loss:  0.557647
Step:  1400 . Loss:  0.0768923
Step:  1500 . Loss:  1.59089
Step:  1600 . Loss:  0.0368046
Step:  1700 . Loss:  0.637939
Step:  1800 . Loss:  0.403258
Step:  1900 . Loss:  0.276448


In [19]:
val_batch = sess.run(val_input)
acc = sess.run(accuracy, feed_dict={X: val_batch[0], target_Class:val_batch[1], is_training: False})
print acc

0.921061


# Dropout = 0.75 (first layer), lr = 5e-3

In [27]:
for i in range(max_steps):
    
    train_batch = sess.run(train_input)
    summary_str, _ = sess.run([summary_op, train_step], feed_dict={
            X:train_batch[0], target_Class:train_batch[1], is_training: True})
    summary_writer.add_summary(summary_str, i)    
    if i % 100 == 0:
        loss = sess.run(cross_entropy,feed_dict={X:train_batch[0], target_Class:train_batch[1], is_training: False})
        print 'Step: ',i,'. Loss: ',loss
        #if loss < 0.05 :
        #    saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))
        #    break
    if i % 1000 == 0 or (i + 1) == max_steps:
        saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))

Step:  0 . Loss:  4.16728
Step:  100 . Loss:  3.32084
Step:  200 . Loss:  1.48668
Step:  300 . Loss:  0.863478
Step:  400 . Loss:  0.540382
Step:  500 . Loss:  0.0998179
Step:  600 . Loss:  0.46698
Step:  700 . Loss:  0.132601
Step:  800 . Loss:  0.218672
Step:  900 . Loss:  0.347927
Step:  1000 . Loss:  0.156757
Step:  1100 . Loss:  0.169492
Step:  1200 . Loss:  0.143467
Step:  1300 . Loss:  0.177369
Step:  1400 . Loss:  0.115933
Step:  1500 . Loss:  0.0236459
Step:  1600 . Loss:  0.0464055
Step:  1700 . Loss:  0.0610753
Step:  1800 . Loss:  0.0366021
Step:  1900 . Loss:  0.00118205


In [28]:
val_batch = sess.run(val_input)
acc = sess.run(accuracy, feed_dict={X: val_batch[0], target_Class:val_batch[1], is_training: False})
print acc

0.896912


# Dropout = 0.50 (first layer), lr = 1e-3

In [29]:
for i in range(max_steps):
    
    train_batch = sess.run(train_input)
    summary_str, _ = sess.run([summary_op, train_step], feed_dict={
            X:train_batch[0], target_Class:train_batch[1], is_training: True})
    summary_writer.add_summary(summary_str, i)    
    if i % 100 == 0:
        loss = sess.run(cross_entropy,feed_dict={X:train_batch[0], target_Class:train_batch[1], is_training: False})
        print 'Step: ',i,'. Loss: ',loss
        #if loss < 0.05 :
        #    saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))
        #    break
    if i % 1000 == 0 or (i + 1) == max_steps:
        saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))

Step:  0 . Loss:  3.2954
Step:  100 . Loss:  1.36617
Step:  200 . Loss:  1.03301
Step:  300 . Loss:  0.946421
Step:  400 . Loss:  0.659636
Step:  500 . Loss:  0.613479
Step:  600 . Loss:  0.506634
Step:  700 . Loss:  0.373168
Step:  800 . Loss:  0.295519
Step:  900 . Loss:  0.199702
Step:  1000 . Loss:  0.23973
Step:  1100 . Loss:  0.208186
Step:  1200 . Loss:  0.0853827
Step:  1300 . Loss:  0.105373
Step:  1400 . Loss:  0.111231
Step:  1500 . Loss:  0.0798168
Step:  1600 . Loss:  0.0409815
Step:  1700 . Loss:  0.0751088
Step:  1800 . Loss:  0.0463233
Step:  1900 . Loss:  0.0466431


In [30]:
val_batch = sess.run(val_input)
acc = sess.run(accuracy, feed_dict={X: val_batch[0], target_Class:val_batch[1], is_training: False})
print acc

0.914727


# Dropout = 0.50 (first layer), lr = 5e-3

In [34]:
for i in range(max_steps):
    
    train_batch = sess.run(train_input)
    summary_str, _ = sess.run([summary_op, train_step], feed_dict={
            X:train_batch[0], target_Class:train_batch[1], is_training: True})
    summary_writer.add_summary(summary_str, i)    
    if i % 100 == 0:
        loss = sess.run(cross_entropy,feed_dict={X:train_batch[0], target_Class:train_batch[1], is_training: False})
        print 'Step: ',i,'. Loss: ',loss
        #if loss < 0.05 :
        #    saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))
        #    break
    if i % 1000 == 0 or (i + 1) == max_steps:
        saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))

Step:  0 . Loss:  6.62296
Step:  100 . Loss:  5.79374
Step:  200 . Loss:  1.21926
Step:  300 . Loss:  1.59369
Step:  400 . Loss:  1.14089
Step:  500 . Loss:  1.37028
Step:  600 . Loss:  0.845161
Step:  700 . Loss:  0.564346
Step:  800 . Loss:  0.198681
Step:  900 . Loss:  0.0883338
Step:  1000 . Loss:  0.265645
Step:  1100 . Loss:  0.534242
Step:  1200 . Loss:  0.00857698
Step:  1300 . Loss:  0.179123
Step:  1400 . Loss:  0.146268
Step:  1500 . Loss:  0.139928
Step:  1600 . Loss:  0.134172
Step:  1700 . Loss:  0.153779
Step:  1800 . Loss:  0.0746156
Step:  1900 . Loss:  0.19611


In [35]:
val_batch = sess.run(val_input)
acc = sess.run(accuracy, feed_dict={X: val_batch[0], target_Class:val_batch[1], is_training: False})
print acc

0.894299


# Dropout = 0.25 (first layer), lr = 1e-3

In [41]:
for i in range(max_steps):
    
    train_batch = sess.run(train_input)
    summary_str, _ = sess.run([summary_op, train_step], feed_dict={
            X:train_batch[0], target_Class:train_batch[1], is_training: True})
    summary_writer.add_summary(summary_str, i)    
    if i % 100 == 0:
        loss = sess.run(cross_entropy,feed_dict={X:train_batch[0], target_Class:train_batch[1], is_training: False})
        print 'Step: ',i,'. Loss: ',loss
        #if loss < 0.05 :
        #    saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))
        #    break
    if i % 1000 == 0 or (i + 1) == max_steps:
        saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))

Step:  0 . Loss:  3.84739
Step:  100 . Loss:  2.60637
Step:  200 . Loss:  1.21422
Step:  300 . Loss:  0.614812
Step:  400 . Loss:  0.647092
Step:  500 . Loss:  0.431192
Step:  600 . Loss:  0.382036
Step:  700 . Loss:  0.245916
Step:  800 . Loss:  0.202527
Step:  900 . Loss:  0.19261
Step:  1000 . Loss:  0.224916
Step:  1100 . Loss:  0.136206
Step:  1200 . Loss:  0.152422
Step:  1300 . Loss:  0.0749793
Step:  1400 . Loss:  0.140265
Step:  1500 . Loss:  0.0913039
Step:  1600 . Loss:  0.186984
Step:  1700 . Loss:  0.12937
Step:  1800 . Loss:  0.102233
Step:  1900 . Loss:  0.0457983


In [42]:
val_batch = sess.run(val_input)
acc = sess.run(accuracy, feed_dict={X: val_batch[0], target_Class:val_batch[1], is_training: False})
print acc

0.887965


# Dropout = 0.25 (first layer), lr = 1e-4

In [13]:
for i in range(max_steps):
    
    train_batch = sess.run(train_input)
    summary_str, _ = sess.run([summary_op, train_step], feed_dict={
            X:train_batch[0], target_Class:train_batch[1], is_training: True})
    summary_writer.add_summary(summary_str, i)    
    if i % 100 == 0:
        loss = sess.run(cross_entropy,feed_dict={X:train_batch[0], target_Class:train_batch[1], is_training: False})
        print 'Step: ',i,'. Loss: ',loss
        #if loss < 0.05 :
        #    saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))
        #    break
    if i % 1000 == 0 or (i + 1) == max_steps:
        saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))

Step:  0 . Loss:  4.56221
Step:  100 . Loss:  3.50385
Step:  200 . Loss:  2.99552
Step:  300 . Loss:  2.43411
Step:  400 . Loss:  2.09392
Step:  500 . Loss:  1.53512
Step:  600 . Loss:  1.65774
Step:  700 . Loss:  1.52918
Step:  800 . Loss:  1.34618
Step:  900 . Loss:  1.13438
Step:  1000 . Loss:  1.24701
Step:  1100 . Loss:  1.23941
Step:  1200 . Loss:  1.00211
Step:  1300 . Loss:  0.92557
Step:  1400 . Loss:  0.743873
Step:  1500 . Loss:  0.84613
Step:  1600 . Loss:  0.752264
Step:  1700 . Loss:  0.75734
Step:  1800 . Loss:  0.641819
Step:  1900 . Loss:  0.720194


In [14]:
val_batch = sess.run(val_input)
acc = sess.run(accuracy, feed_dict={X: val_batch[0], target_Class:val_batch[1], is_training: False})
print acc

0.750515


# Dropout = 0.25 (all three layers), lr = 1e-3

In [19]:
for i in range(max_steps):
    
    train_batch = sess.run(train_input)
    summary_str, _ = sess.run([summary_op, train_step], feed_dict={
            X:train_batch[0], target_Class:train_batch[1], is_training: True})
    summary_writer.add_summary(summary_str, i)    
    if i % 100 == 0:
        loss = sess.run(cross_entropy,feed_dict={X:train_batch[0], target_Class:train_batch[1], is_training: False})
        print 'Step: ',i,'. Loss: ',loss
        #if loss < 0.05 :
        #    saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))
        #    break
    if i % 1000 == 0 or (i + 1) == max_steps:
        saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))

Step:  0 . Loss:  16.1139
Step:  100 . Loss:  11.2872
Step:  200 . Loss:  6.98185
Step:  300 . Loss:  5.0678
Step:  400 . Loss:  3.36569
Step:  500 . Loss:  2.26278
Step:  600 . Loss:  2.70454
Step:  700 . Loss:  2.27804
Step:  800 . Loss:  1.48459
Step:  900 . Loss:  1.36311
Step:  1000 . Loss:  1.67648
Step:  1100 . Loss:  1.11311
Step:  1200 . Loss:  0.99523
Step:  1300 . Loss:  1.28798
Step:  1400 . Loss:  1.08535
Step:  1500 . Loss:  1.07099
Step:  1600 . Loss:  0.820302
Step:  1700 . Loss:  0.961997
Step:  1800 . Loss:  0.967935
Step:  1900 . Loss:  0.94755


In [20]:
val_batch = sess.run(val_input)
acc = sess.run(accuracy, feed_dict={X: val_batch[0], target_Class:val_batch[1], is_training: False})
print acc

0.648219


# Dropout = 0.75 (last layer), lr = 1e-3

In [25]:
for i in range(max_steps):
    
    train_batch = sess.run(train_input)
    summary_str, _ = sess.run([summary_op, train_step], feed_dict={
            X:train_batch[0], target_Class:train_batch[1], is_training: True})
    summary_writer.add_summary(summary_str, i)    
    if i % 100 == 0:
        loss = sess.run(cross_entropy,feed_dict={X:train_batch[0], target_Class:train_batch[1], is_training: False})
        print 'Step: ',i,'. Loss: ',loss
        #if loss < 0.05 :
        #    saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))
        #    break
    if i % 1000 == 0 or (i + 1) == max_steps:
        saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))

Step:  0 . Loss:  3.12127
Step:  100 . Loss:  2.09219
Step:  200 . Loss:  2.14728
Step:  300 . Loss:  1.93018
Step:  400 . Loss:  1.55527
Step:  500 . Loss:  1.32356
Step:  600 . Loss:  1.21193
Step:  700 . Loss:  0.739841
Step:  800 . Loss:  0.639243
Step:  900 . Loss:  0.510116
Step:  1000 . Loss:  0.407085
Step:  1100 . Loss:  0.226974
Step:  1200 . Loss:  0.242082
Step:  1300 . Loss:  0.139449
Step:  1400 . Loss:  0.119541
Step:  1500 . Loss:  0.066762
Step:  1600 . Loss:  0.0982869
Step:  1700 . Loss:  0.130275
Step:  1800 . Loss:  0.0612288
Step:  1900 . Loss:  0.0268217


In [26]:
val_batch = sess.run(val_input)
acc = sess.run(accuracy, feed_dict={X: val_batch[0], target_Class:val_batch[1], is_training: False})
print acc

0.911322


# Dropout = 0.75 (middle layer), lr = 1e-3

In [37]:
for i in range(max_steps):
    
    train_batch = sess.run(train_input)
    summary_str, _ = sess.run([summary_op, train_step], feed_dict={
            X:train_batch[0], target_Class:train_batch[1], is_training: True})
    summary_writer.add_summary(summary_str, i)    
    if i % 100 == 0:
        loss = sess.run(cross_entropy,feed_dict={X:train_batch[0], target_Class:train_batch[1], is_training: False})
        print 'Step: ',i,'. Loss: ',loss
        #if loss < 0.05 :
        #    saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))
        #    break
    if i % 1000 == 0 or (i + 1) == max_steps:
        saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))

Step:  0 . Loss:  3.24211
Step:  100 . Loss:  1.96572
Step:  200 . Loss:  1.7286
Step:  300 . Loss:  1.34055
Step:  400 . Loss:  1.22073
Step:  500 . Loss:  0.868452
Step:  600 . Loss:  0.745769
Step:  700 . Loss:  0.669053
Step:  800 . Loss:  0.446787
Step:  900 . Loss:  0.400223
Step:  1000 . Loss:  0.411238
Step:  1100 . Loss:  0.271472
Step:  1200 . Loss:  0.237348
Step:  1300 . Loss:  0.124074
Step:  1400 . Loss:  0.127163
Step:  1500 . Loss:  0.113435
Step:  1600 . Loss:  0.0457504
Step:  1700 . Loss:  0.0612575
Step:  1800 . Loss:  0.063899
Step:  1900 . Loss:  0.0323914


In [38]:
val_batch = sess.run(val_input)
acc = sess.run(accuracy, feed_dict={X: val_batch[0], target_Class:val_batch[1], is_training: False})
print acc

0.908393


# Dropout = 0.9 (first layer), lr = 1e-3

In [43]:
for i in range(max_steps):
    
    train_batch = sess.run(train_input)
    summary_str, _ = sess.run([summary_op, train_step], feed_dict={
            X:train_batch[0], target_Class:train_batch[1], is_training: True})
    summary_writer.add_summary(summary_str, i)    
    if i % 100 == 0:
        loss = sess.run(cross_entropy,feed_dict={X:train_batch[0], target_Class:train_batch[1], is_training: False})
        print 'Step: ',i,'. Loss: ',loss
        #if loss < 0.05 :
        #    saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))
        #    break
    if i % 1000 == 0 or (i + 1) == max_steps:
        saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))

Step:  0 . Loss:  3.19643
Step:  100 . Loss:  2.20663
Step:  200 . Loss:  1.98418
Step:  300 . Loss:  1.56505
Step:  400 . Loss:  1.3687
Step:  500 . Loss:  1.14797
Step:  600 . Loss:  0.909471
Step:  700 . Loss:  0.677546
Step:  800 . Loss:  0.650981
Step:  900 . Loss:  0.492434
Step:  1000 . Loss:  0.416482
Step:  1100 . Loss:  0.245484
Step:  1200 . Loss:  0.183636
Step:  1300 . Loss:  0.127822
Step:  1400 . Loss:  0.10856
Step:  1500 . Loss:  0.0702929
Step:  1600 . Loss:  0.105716
Step:  1700 . Loss:  0.0922665
Step:  1800 . Loss:  0.0544483
Step:  1900 . Loss:  0.0239642


In [44]:
val_batch = sess.run(val_input)
acc = sess.run(accuracy, feed_dict={X: val_batch[0], target_Class:val_batch[1], is_training: False})
print acc

0.919319


# Dropout = 0.5 (last layer), lr = 1e-3

In [63]:
for i in range(max_steps):
    
    train_batch = sess.run(train_input)
    summary_str, _ = sess.run([summary_op, train_step], feed_dict={
            X:train_batch[0], target_Class:train_batch[1], is_training: True})
    summary_writer.add_summary(summary_str, i)    
    if i % 100 == 0:
        loss = sess.run(cross_entropy,feed_dict={X:train_batch[0], target_Class:train_batch[1], is_training: False})
        print 'Step: ',i,'. Loss: ',loss
        #if loss < 0.05 :
        #    saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))
        #    break
    if i % 1000 == 0 or (i + 1) == max_steps:
        saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))

Step:  0 . Loss:  3.55842
Step:  100 . Loss:  3.68531
Step:  200 . Loss:  3.5714
Step:  300 . Loss:  3.63173
Step:  400 . Loss:  3.56067
Step:  500 . Loss:  3.26847
Step:  600 . Loss:  3.30251
Step:  700 . Loss:  3.32587
Step:  800 . Loss:  3.37492
Step:  900 . Loss:  3.10906
Step:  1000 . Loss:  3.14536
Step:  1100 . Loss:  2.92082
Step:  1200 . Loss:  2.97112
Step:  1300 . Loss:  2.57476
Step:  1400 . Loss:  2.88524
Step:  1500 . Loss:  2.95434
Step:  1600 . Loss:  3.02012
Step:  1700 . Loss:  2.75022
Step:  1800 . Loss:  3.07213
Step:  1900 . Loss:  2.82448


In [64]:
val_batch = sess.run(val_input)
acc = sess.run(accuracy, feed_dict={X: val_batch[0], target_Class:val_batch[1], is_training: False})
print acc

0.233017


# Dropout = 0.75 (first layer), lr = 1e-3 (4000 steps)

In [36]:
for i in range(max_steps):
    
    train_batch = sess.run(train_input)
    summary_str, _ = sess.run([summary_op, train_step], feed_dict={
            X:train_batch[0], target_Class:train_batch[1], is_training: True})
    summary_writer.add_summary(summary_str, i)    
    if i % 100 == 0:
        loss = sess.run(cross_entropy,feed_dict={X:train_batch[0], target_Class:train_batch[1], is_training: False})
        print 'Step: ',i,'. Loss: ',loss
        #if loss < 0.05 :
        #    saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))
        #    break
    if i % 1000 == 0 or (i + 1) == max_steps:
        saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))

Step:  0 . Loss:  3.16996
Step:  100 . Loss:  2.23956
Step:  200 . Loss:  1.7717
Step:  300 . Loss:  1.4637
Step:  400 . Loss:  1.603
Step:  500 . Loss:  1.00123
Step:  600 . Loss:  0.846839
Step:  700 . Loss:  0.684138
Step:  800 . Loss:  0.697838
Step:  900 . Loss:  0.366344
Step:  1000 . Loss:  0.26925
Step:  1100 . Loss:  0.272609
Step:  1200 . Loss:  0.209791
Step:  1300 . Loss:  0.126413
Step:  1400 . Loss:  0.0723379
Step:  1500 . Loss:  0.0670332
Step:  1600 . Loss:  0.0669736
Step:  1700 . Loss:  0.0647514
Step:  1800 . Loss:  0.033221
Step:  1900 . Loss:  0.0362971
Step:  2000 . Loss:  0.0123967
Step:  2100 . Loss:  0.0395519
Step:  2200 . Loss:  0.0169982
Step:  2300 . Loss:  0.0367263
Step:  2400 . Loss:  0.00842539
Step:  2500 . Loss:  0.0170316
Step:  2600 . Loss:  0.018728
Step:  2700 . Loss:  0.00468952
Step:  2800 . Loss:  0.0257063
Step:  2900 . Loss:  0.0078549
Step:  3000 . Loss:  0.0126013
Step:  3100 . Loss:  0.00937159
Step:  3200 . Loss:  0.0037049
Step:  3300 .

In [37]:
val_batch = sess.run(val_input)
acc = sess.run(accuracy, feed_dict={X: val_batch[0], target_Class:val_batch[1], is_training: False})
print acc

0.923357


# No Regularization, lr = 1e-3 (4000 steps)

In [42]:
for i in range(max_steps):
    
    train_batch = sess.run(train_input)
    summary_str, _ = sess.run([summary_op, train_step], feed_dict={
            X:train_batch[0], target_Class:train_batch[1], is_training: True})
    summary_writer.add_summary(summary_str, i)    
    if i % 100 == 0:
        loss = sess.run(cross_entropy,feed_dict={X:train_batch[0], target_Class:train_batch[1], is_training: False})
        print 'Step: ',i,'. Loss: ',loss
        #if loss < 0.05 :
        #    saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))
        #    break
    if i % 1000 == 0 or (i + 1) == max_steps:
        saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))

Step:  0 . Loss:  3.27126
Step:  100 . Loss:  2.7425
Step:  200 . Loss:  2.50846
Step:  300 . Loss:  2.28948
Step:  400 . Loss:  1.81827
Step:  500 . Loss:  1.53847
Step:  600 . Loss:  1.39143
Step:  700 . Loss:  0.923713
Step:  800 . Loss:  0.631433
Step:  900 . Loss:  0.615287
Step:  1000 . Loss:  0.40281
Step:  1100 . Loss:  0.371273
Step:  1200 . Loss:  0.18481
Step:  1300 . Loss:  0.194823
Step:  1400 . Loss:  0.122308
Step:  1500 . Loss:  0.131396
Step:  1600 . Loss:  0.0362113
Step:  1700 . Loss:  0.0576668
Step:  1800 . Loss:  0.028953
Step:  1900 . Loss:  0.0317863
Step:  2000 . Loss:  0.0239923
Step:  2100 . Loss:  0.0127741
Step:  2200 . Loss:  0.0102238
Step:  2300 . Loss:  0.0169185
Step:  2400 . Loss:  0.0164253
Step:  2500 . Loss:  0.00831673
Step:  2600 . Loss:  0.00916131
Step:  2700 . Loss:  0.00656087
Step:  2800 . Loss:  0.00431385
Step:  2900 . Loss:  0.00348907
Step:  3000 . Loss:  0.0031575
Step:  3100 . Loss:  0.00428604
Step:  3200 . Loss:  0.00215819
Step:  33

In [43]:
val_batch = sess.run(val_input)
acc = sess.run(accuracy, feed_dict={X: val_batch[0], target_Class:val_batch[1], is_training: False})
print acc

0.939509


# Dropout = 0.5 (last layer), lr = 1e-3

In [13]:
for i in range(max_steps):
    
    train_batch = sess.run(train_input)
    summary_str, _ = sess.run([summary_op, train_step], feed_dict={
            X:train_batch[0], target_Class:train_batch[1], is_training: True})
    summary_writer.add_summary(summary_str, i)    
    if i % 100 == 0:
        loss = sess.run(cross_entropy,feed_dict={X:train_batch[0], target_Class:train_batch[1], is_training: False})
        print 'Step: ',i,'. Loss: ',loss
        #if loss < 0.05 :
        #    saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))
        #    break
    if i % 1000 == 0 or (i + 1) == max_steps:
        saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))

Step:  0 . Loss:  3.20708
Step:  100 . Loss:  1.69797
Step:  200 . Loss:  1.61656
Step:  300 . Loss:  1.45878
Step:  400 . Loss:  1.44049
Step:  500 . Loss:  1.23969
Step:  600 . Loss:  0.885668
Step:  700 . Loss:  0.726471
Step:  800 . Loss:  0.620086
Step:  900 . Loss:  0.40522
Step:  1000 . Loss:  0.260088
Step:  1100 . Loss:  0.187275
Step:  1200 . Loss:  0.233846
Step:  1300 . Loss:  0.176295
Step:  1400 . Loss:  0.162203
Step:  1500 . Loss:  0.155508
Step:  1600 . Loss:  0.11534
Step:  1700 . Loss:  0.0846709
Step:  1800 . Loss:  0.0832071
Step:  1900 . Loss:  0.0597158


In [14]:
val_batch = sess.run(val_input)
acc = sess.run(accuracy, feed_dict={X: val_batch[0], target_Class:val_batch[1], is_training: False})
print acc

0.907284


# Dropout = 0.5 (middle layer), lr = 1e-3

In [19]:
for i in range(max_steps):
    
    train_batch = sess.run(train_input)
    summary_str, _ = sess.run([summary_op, train_step], feed_dict={
            X:train_batch[0], target_Class:train_batch[1], is_training: True})
    summary_writer.add_summary(summary_str, i)    
    if i % 100 == 0:
        loss = sess.run(cross_entropy,feed_dict={X:train_batch[0], target_Class:train_batch[1], is_training: False})
        print 'Step: ',i,'. Loss: ',loss
        #if loss < 0.05 :
        #    saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))
        #    break
    if i % 1000 == 0 or (i + 1) == max_steps:
        saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))

Step:  0 . Loss:  3.25864
Step:  100 . Loss:  1.95356
Step:  200 . Loss:  1.514
Step:  300 . Loss:  1.18764
Step:  400 . Loss:  0.956713
Step:  500 . Loss:  0.746332
Step:  600 . Loss:  0.546943
Step:  700 . Loss:  0.408633
Step:  800 . Loss:  0.365197
Step:  900 . Loss:  0.198042
Step:  1000 . Loss:  0.205343
Step:  1100 . Loss:  0.210917
Step:  1200 . Loss:  0.171875
Step:  1300 . Loss:  0.126247
Step:  1400 . Loss:  0.107294
Step:  1500 . Loss:  0.115601
Step:  1600 . Loss:  0.0430274
Step:  1700 . Loss:  0.0559825
Step:  1800 . Loss:  0.0376206
Step:  1900 . Loss:  0.0573068


In [20]:
val_batch = sess.run(val_input)
acc = sess.run(accuracy, feed_dict={X: val_batch[0], target_Class:val_batch[1], is_training: False})
print acc

0.907601


# Dropout = 0.5 (middle layer), lr = 5e-3

In [24]:
for i in range(max_steps):
    
    train_batch = sess.run(train_input)
    summary_str, _ = sess.run([summary_op, train_step], feed_dict={
            X:train_batch[0], target_Class:train_batch[1], is_training: True})
    summary_writer.add_summary(summary_str, i)    
    if i % 100 == 0:
        loss = sess.run(cross_entropy,feed_dict={X:train_batch[0], target_Class:train_batch[1], is_training: False})
        print 'Step: ',i,'. Loss: ',loss
        #if loss < 0.05 :
        #    saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))
        #    break
    if i % 1000 == 0 or (i + 1) == max_steps:
        saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))

Step:  0 . Loss:  6.27248
Step:  100 . Loss:  4.33184
Step:  200 . Loss:  1.61654
Step:  300 . Loss:  1.03965
Step:  400 . Loss:  1.34462
Step:  500 . Loss:  0.595149
Step:  600 . Loss:  0.436976
Step:  700 . Loss:  0.479226
Step:  800 . Loss:  0.295557
Step:  900 . Loss:  0.728844
Step:  1000 . Loss:  0.375222
Step:  1100 . Loss:  0.303997
Step:  1200 . Loss:  0.289826
Step:  1300 . Loss:  0.228252
Step:  1400 . Loss:  0.507276
Step:  1500 . Loss:  0.134793
Step:  1600 . Loss:  0.103756
Step:  1700 . Loss:  0.484255
Step:  1800 . Loss:  0.225907
Step:  1900 . Loss:  0.142048


In [25]:
val_batch = sess.run(val_input)
acc = sess.run(accuracy, feed_dict={X: val_batch[0], target_Class:val_batch[1], is_training: False})
print acc

0.875772


# Hinge Loss without Regularization (lr = 1e-3)

In [20]:
for i in range(max_steps):
    
    train_batch = sess.run(train_input)
    summary_str, _ = sess.run([summary_op, train_step], feed_dict={
            X:train_batch[0], target_Class:train_batch[1], is_training: True})
    summary_writer.add_summary(summary_str, i)    
    if i % 100 == 0:
        
        acc = sess.run(accuracy, feed_dict={X: val_batch[0], target_Class:val_batch[1], is_training: False})
        
        loss = sess.run(final_loss,feed_dict={X:train_batch[0], target_Class:train_batch[1], is_training: False})
        print 'Step: ',i,'. Loss: ',loss,'Accuracy: ',acc
        
        #if loss < 0.05 :
        #    saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))
        #    break
    if i % 1000 == 0 or (i + 1) == max_steps:
        saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))

Step:  0 . Loss:  0.118169 Accuracy:  0.0159145
Step:  100 . Loss:  0.0994305 Accuracy:  0.122328
Step:  200 . Loss:  0.0567423 Accuracy:  0.409026
Step:  300 . Loss:  0.0495981 Accuracy:  0.571338
Step:  400 . Loss:  0.10735 Accuracy:  0.71639
Step:  500 . Loss:  0.120744 Accuracy:  0.787569
Step:  600 . Loss:  0.103508 Accuracy:  0.83753
Step:  700 . Loss:  0.105103 Accuracy:  0.83175
Step:  800 . Loss:  0.0559446 Accuracy:  0.855344
Step:  900 . Loss:  0.054427 Accuracy:  0.866588
Step:  1000 . Loss:  0.043141 Accuracy:  0.869438
Step:  1100 . Loss:  0.0483084 Accuracy:  0.888836
Step:  1200 . Loss:  0.013869 Accuracy:  0.883848
Step:  1300 . Loss:  0.0210285 Accuracy:  0.90285
Step:  1400 . Loss:  0.00999276 Accuracy:  0.912589
Step:  1500 . Loss:  0.00630162 Accuracy:  0.905463
Step:  1600 . Loss:  0.0113905 Accuracy:  0.917498
Step:  1700 . Loss:  0.00427542 Accuracy:  0.913143
Step:  1800 . Loss:  0.00482325 Accuracy:  0.922011
Step:  1900 . Loss:  0.00702487 Accuracy:  0.925336

In [21]:
val_batch = sess.run(val_input)
acc = sess.run(accuracy, feed_dict={X: val_batch[0], target_Class:val_batch[1], is_training: False})
print acc

0.921774


# Hinge Loss with Regularization on all layers (lr = 1e-3, lambda = 1e-4)

In [25]:
for i in range(max_steps):
    
    train_batch = sess.run(train_input)
    summary_str, _ = sess.run([summary_op, train_step], feed_dict={
            X:train_batch[0], target_Class:train_batch[1], is_training: True})
    summary_writer.add_summary(summary_str, i)    
    if i % 100 == 0:
        
        acc = sess.run(accuracy, feed_dict={X: val_batch[0], target_Class:val_batch[1], is_training: False})
        
        loss = sess.run(final_loss,feed_dict={X:train_batch[0], target_Class:train_batch[1], is_training: False})
        print 'Step: ',i,'. Loss: ',loss,'Accuracy: ',acc
        
        #if loss < 0.05 :
        #    saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))
        #    break
    if i % 1000 == 0 or (i + 1) == max_steps:
        saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))

Step:  0 . Loss:  0.122106 Accuracy:  0.0214568
Step:  100 . Loss:  0.0800602 Accuracy:  0.200871
Step:  200 . Loss:  0.127177 Accuracy:  0.397229
Step:  300 . Loss:  0.369744 Accuracy:  0.620823
Step:  400 . Loss:  0.367617 Accuracy:  0.754869
Step:  500 . Loss:  0.342103 Accuracy:  0.78361
Step:  600 . Loss:  0.398549 Accuracy:  0.836184
Step:  700 . Loss:  0.310535 Accuracy:  0.847981
Step:  800 . Loss:  0.312676 Accuracy:  0.868329
Step:  900 . Loss:  0.280864 Accuracy:  0.858512
Step:  1000 . Loss:  0.263061 Accuracy:  0.883769
Step:  1100 . Loss:  0.158154 Accuracy:  0.895091
Step:  1200 . Loss:  0.116719 Accuracy:  0.895804
Step:  1300 . Loss:  0.0835526 Accuracy:  0.914252
Step:  1400 . Loss:  0.0654995 Accuracy:  0.906968
Step:  1500 . Loss:  0.0589969 Accuracy:  0.907363
Step:  1600 . Loss:  0.0648629 Accuracy:  0.909264
Step:  1700 . Loss:  0.030619 Accuracy:  0.920348
Step:  1800 . Loss:  0.0373643 Accuracy:  0.91829
Step:  1900 . Loss:  0.0243933 Accuracy:  0.921457


In [26]:
val_batch = sess.run(val_input)
acc = sess.run(accuracy, feed_dict={X: val_batch[0], target_Class:val_batch[1], is_training: False})
print acc

0.916152


# Hinge Loss with Regularization on only fully connected layer (lr = 1e-3, lambda = 1e-4)

In [30]:
for i in range(max_steps):
    
    train_batch = sess.run(train_input)
    summary_str, _ = sess.run([summary_op, train_step], feed_dict={
            X:train_batch[0], target_Class:train_batch[1], is_training: True})
    summary_writer.add_summary(summary_str, i)    
    if i % 100 == 0:
        loss = sess.run(final_loss,feed_dict={X:train_batch[0], target_Class:train_batch[1], is_training: False})
        print 'Step: ',i,'. Loss: ',loss
        
        #if loss < 0.05 :
        #    saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))
        #    break
    if i % 1000 == 0 or (i + 1) == max_steps:
        saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))

Step:  0 . Loss:  0.115071
Step:  100 . Loss:  0.100509
Step:  200 . Loss:  0.0520454
Step:  300 . Loss:  0.0529256
Step:  400 . Loss:  0.0978769
Step:  500 . Loss:  0.122575
Step:  600 . Loss:  0.104214
Step:  700 . Loss:  0.0671537
Step:  800 . Loss:  0.0441727
Step:  900 . Loss:  0.0564359
Step:  1000 . Loss:  0.035203
Step:  1100 . Loss:  0.0211783
Step:  1200 . Loss:  0.0203727
Step:  1300 . Loss:  0.0160817
Step:  1400 . Loss:  0.0113473
Step:  1500 . Loss:  0.00701093
Step:  1600 . Loss:  0.00690741
Step:  1700 . Loss:  0.00659867
Step:  1800 . Loss:  0.00686619
Step:  1900 . Loss:  0.00477867


In [31]:
val_batch = sess.run(val_input)
acc = sess.run(accuracy, feed_dict={X: val_batch[0], target_Class:val_batch[1], is_training: False})
print acc

0.918369


# Hinge Loss with Regularization on all layer (lr = 1e-3, lambda = 5e-4)

In [13]:
for i in range(max_steps):
    
    train_batch = sess.run(train_input)
    summary_str, _ = sess.run([summary_op, train_step], feed_dict={
            X:train_batch[0], target_Class:train_batch[1], is_training: True})
    summary_writer.add_summary(summary_str, i)    
    if i % 100 == 0:
        loss = sess.run(final_loss,feed_dict={X:train_batch[0], target_Class:train_batch[1], is_training: False})
        print 'Step: ',i,'. Loss: ',loss
        
        #if loss < 0.05 :
        #    saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))
        #    break
    if i % 1000 == 0 or (i + 1) == max_steps:
        saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))

Step:  0 . Loss:  0.156111
Step:  100 . Loss:  0.089378
Step:  200 . Loss:  0.0535664
Step:  300 . Loss:  0.149591
Step:  400 . Loss:  0.173702
Step:  500 . Loss:  0.25296
Step:  600 . Loss:  0.207785
Step:  700 . Loss:  0.172098
Step:  800 . Loss:  0.113136
Step:  900 . Loss:  0.0956571
Step:  1000 . Loss:  0.0488638
Step:  1100 . Loss:  0.0508386
Step:  1200 . Loss:  0.0342766
Step:  1300 . Loss:  0.0285713
Step:  1400 . Loss:  0.0196951
Step:  1500 . Loss:  0.0185723
Step:  1600 . Loss:  0.0170751
Step:  1700 . Loss:  0.00995203
Step:  1800 . Loss:  0.0114049
Step:  1900 . Loss:  0.0129062


In [14]:
val_batch = sess.run(val_input)
acc = sess.run(accuracy, feed_dict={X: val_batch[0], target_Class:val_batch[1], is_training: False})
print acc

0.933333


# Hinge Loss with Regularization on only fully connected layer (lr = 1e-3, lambda = 5e-4)

In [18]:
for i in range(max_steps):
    
    train_batch = sess.run(train_input)
    summary_str, _ = sess.run([summary_op, train_step], feed_dict={
            X:train_batch[0], target_Class:train_batch[1], is_training: True})
    summary_writer.add_summary(summary_str, i)    
    if i % 100 == 0:
        loss = sess.run(final_loss,feed_dict={X:train_batch[0], target_Class:train_batch[1], is_training: False})
        print 'Step: ',i,'. Loss: ',loss
        
        #if loss < 0.05 :
        #    saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))
        #    break
    if i % 1000 == 0 or (i + 1) == max_steps:
        saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))

Step:  0 . Loss:  0.230314
Step:  100 . Loss:  0.106894
Step:  200 . Loss:  0.637119
Step:  300 . Loss:  0.90321
Step:  400 . Loss:  1.02441
Step:  500 . Loss:  1.06069
Step:  600 . Loss:  0.995734
Step:  700 . Loss:  1.00458
Step:  800 . Loss:  0.904947
Step:  900 . Loss:  0.864348
Step:  1000 . Loss:  0.806455
Step:  1100 . Loss:  0.716547
Step:  1200 . Loss:  0.643991
Step:  1300 . Loss:  0.660114
Step:  1400 . Loss:  0.549792
Step:  1500 . Loss:  0.520844
Step:  1600 . Loss:  0.493345
Step:  1700 . Loss:  0.437093
Step:  1800 . Loss:  0.384636
Step:  1900 . Loss:  0.242769


In [19]:
val_batch = sess.run(val_input)
acc = sess.run(accuracy, feed_dict={X: val_batch[0], target_Class:val_batch[1], is_training: False})
print acc

0.882502


# Hinge Loss no Regularization (lr = 5e-3)

In [13]:
for i in range(max_steps):
    
    train_batch = sess.run(train_input)
    summary_str, _ = sess.run([summary_op, train_step], feed_dict={
            X:train_batch[0], target_Class:train_batch[1], is_training: True})
    summary_writer.add_summary(summary_str, i)    
    if i % 100 == 0:
        loss = sess.run(final_loss,feed_dict={X:train_batch[0], target_Class:train_batch[1], is_training: False})
        print 'Step: ',i,'. Loss: ',loss
        
        #if loss < 0.05 :
        #    saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))
        #    break
    if i % 1000 == 0 or (i + 1) == max_steps:
        saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))

Step:  0 . Loss:  0.185806
Step:  100 . Loss:  0.854869
Step:  200 . Loss:  0.266195
Step:  300 . Loss:  0.114137
Step:  400 . Loss:  0.0691381
Step:  500 . Loss:  0.0463314
Step:  600 . Loss:  0.0290599
Step:  700 . Loss:  0.0236223
Step:  800 . Loss:  0.015579
Step:  900 . Loss:  0.0178797
Step:  1000 . Loss:  0.0253953
Step:  1100 . Loss:  0.0246214
Step:  1200 . Loss:  0.00992134
Step:  1300 . Loss:  0.00929281
Step:  1400 . Loss:  0.00473579
Step:  1500 . Loss:  0.0116638
Step:  1600 . Loss:  0.006737
Step:  1700 . Loss:  0.00554582
Step:  1800 . Loss:  0.00400493
Step:  1900 . Loss:  0.00270115


In [14]:
val_batch = sess.run(val_input)
acc = sess.run(accuracy, feed_dict={X: val_batch[0], target_Class:val_batch[1], is_training: False})
print acc

0.924941


# Hinge Loss with Regularization on fully connected parameters (lr = 5e-3, lambda = 1e-4)

In [13]:
for i in range(max_steps):
    
    train_batch = sess.run(train_input)
    summary_str, _ = sess.run([summary_op, train_step], feed_dict={
            X:train_batch[0], target_Class:train_batch[1], is_training: True})
    summary_writer.add_summary(summary_str, i)    
    if i % 100 == 0:
        loss = sess.run(final_loss,feed_dict={X:train_batch[0], target_Class:train_batch[1], is_training: False})
        print 'Step: ',i,'. Loss: ',loss
        
        #if loss < 0.05 :
        #    saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))
        #    break
    if i % 1000 == 0 or (i + 1) == max_steps:
        saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))

Step:  0 . Loss:  0.212878
Step:  100 . Loss:  0.685353
Step:  200 . Loss:  0.23407
Step:  300 . Loss:  0.0733275
Step:  400 . Loss:  0.0255754
Step:  500 . Loss:  0.0221241
Step:  600 . Loss:  0.0172838
Step:  700 . Loss:  0.016853
Step:  800 . Loss:  0.0178172
Step:  900 . Loss:  0.0107884
Step:  1000 . Loss:  0.0082546
Step:  1100 . Loss:  0.012205
Step:  1200 . Loss:  0.00869569
Step:  1300 . Loss:  0.00682247
Step:  1400 . Loss:  0.00980327
Step:  1500 . Loss:  0.0129191
Step:  1600 . Loss:  0.00732039
Step:  1700 . Loss:  0.00733274
Step:  1800 . Loss:  0.0048902
Step:  1900 . Loss:  0.00707775


In [14]:
val_batch = sess.run(val_input)
acc = sess.run(accuracy, feed_dict={X: val_batch[0], target_Class:val_batch[1], is_training: False})
print acc

0.933175


# Hinge Loss with Regularization on all parameters (lr = 5e-3, lambda = 5e-4

In [13]:
for i in range(max_steps):
    
    train_batch = sess.run(train_input)
    summary_str, _ = sess.run([summary_op, train_step], feed_dict={
            X:train_batch[0], target_Class:train_batch[1], is_training: True})
    summary_writer.add_summary(summary_str, i)    
    if i % 100 == 0:
        loss = sess.run(final_loss,feed_dict={X:train_batch[0], target_Class:train_batch[1], is_training: False})
        print 'Step: ',i,'. Loss: ',loss
        
        #if loss < 0.05 :
        #    saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))
        #    break
    if i % 1000 == 0 or (i + 1) == max_steps:
        saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))

Step:  0 . Loss:  0.279055
Step:  100 . Loss:  0.219018
Step:  200 . Loss:  0.786828
Step:  300 . Loss:  0.989573
Step:  400 . Loss:  1.02419
Step:  500 . Loss:  0.60585
Step:  600 . Loss:  0.558758
Step:  700 . Loss:  0.439166
Step:  800 . Loss:  0.368158
Step:  900 . Loss:  0.2629
Step:  1000 . Loss:  0.262154
Step:  1100 . Loss:  0.18949
Step:  1200 . Loss:  0.227115
Step:  1300 . Loss:  0.404901
Step:  1400 . Loss:  0.250201
Step:  1500 . Loss:  0.0748346
Step:  1600 . Loss:  0.0968797
Step:  1700 . Loss:  0.106577
Step:  1800 . Loss:  0.0957323
Step:  1900 . Loss:  0.0864901


In [14]:
val_batch = sess.run(val_input)
acc = sess.run(accuracy, feed_dict={X: val_batch[0], target_Class:val_batch[1], is_training: False})
print acc

0.775376


# Hinge Loss with Regularization on all parameters (lr = 5e-3, lambda = 1e-4)

In [13]:
for i in range(max_steps):
    
    train_batch = sess.run(train_input)
    summary_str, _ = sess.run([summary_op, train_step], feed_dict={
            X:train_batch[0], target_Class:train_batch[1], is_training: True})
    summary_writer.add_summary(summary_str, i)    
    if i % 100 == 0:
        loss = sess.run(final_loss,feed_dict={X:train_batch[0], target_Class:train_batch[1], is_training: False})
        print 'Step: ',i,'. Loss: ',loss
        
        #if loss < 0.05 :
        #    saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))
        #    break
    if i % 1000 == 0 or (i + 1) == max_steps:
        saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))

Step:  0 . Loss:  0.195729
Step:  100 . Loss:  0.67274
Step:  200 . Loss:  0.122922
Step:  300 . Loss:  0.0522099
Step:  400 . Loss:  0.0595671
Step:  500 . Loss:  0.0419748
Step:  600 . Loss:  0.0612903
Step:  700 . Loss:  0.0343615
Step:  800 . Loss:  0.0514625
Step:  900 . Loss:  0.0261861
Step:  1000 . Loss:  0.032759
Step:  1100 . Loss:  0.0244642
Step:  1200 . Loss:  0.0251186
Step:  1300 . Loss:  0.0219984
Step:  1400 . Loss:  0.0227893
Step:  1500 . Loss:  0.017236
Step:  1600 . Loss:  0.0215277
Step:  1700 . Loss:  0.018411
Step:  1800 . Loss:  0.0205493
Step:  1900 . Loss:  0.0188811


In [14]:
val_batch = sess.run(val_input)
acc = sess.run(accuracy, feed_dict={X: val_batch[0], target_Class:val_batch[1], is_training: False})
print acc

0.870467


# Hinge Loss with Regularization on fully connected parameters (lr = 5e-3, lambda = 5e-4)

In [13]:
for i in range(max_steps):
    
    train_batch = sess.run(train_input)
    summary_str, _ = sess.run([summary_op, train_step], feed_dict={
            X:train_batch[0], target_Class:train_batch[1], is_training: True})
    summary_writer.add_summary(summary_str, i)    
    if i % 100 == 0:
        loss = sess.run(final_loss,feed_dict={X:train_batch[0], target_Class:train_batch[1], is_training: False})
        print 'Step: ',i,'. Loss: ',loss
        
        #if loss < 0.05 :
        #    saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))
        #    break
    if i % 1000 == 0 or (i + 1) == max_steps:
        saver.save(sess, os.path.join(train_dir, 'Save/model.ckpt'))

Step:  0 . Loss:  0.167912
Step:  100 . Loss:  0.4594
Step:  200 . Loss:  0.164285
Step:  300 . Loss:  0.0625068
Step:  400 . Loss:  0.0285088
Step:  500 . Loss:  0.0248637
Step:  600 . Loss:  0.0209271
Step:  700 . Loss:  0.0252341
Step:  800 . Loss:  0.0194386
Step:  900 . Loss:  0.0139315
Step:  1000 . Loss:  0.0194199
Step:  1100 . Loss:  0.018275
Step:  1200 . Loss:  0.0114665
Step:  1300 . Loss:  0.0111969
Step:  1400 . Loss:  0.0144521
Step:  1500 . Loss:  0.0114778
Step:  1600 . Loss:  0.013318
Step:  1700 . Loss:  0.00992228
Step:  1800 . Loss:  0.00800235
Step:  1900 . Loss:  0.008207


In [14]:
val_batch = sess.run(val_input)
acc = sess.run(accuracy, feed_dict={X: val_batch[0], target_Class:val_batch[1], is_training: False})
print acc

0.922249
